In [79]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
import time
import datetime
from IPython import display
import random
import cv2


""" processed_dir = ["0001 - bleach", "0002 - one piece","0003 - marvel","0004 - DC", "0005 - Love", "0006 - soma", 
                        "0007 - ishoujo", "0008 - vagabond", "0009 - vampire r", "0010 - gintama", "0011 - gantz",
                         "0012 - chainsaw", "0013 - death note", "0014 - batman", "0015 - test", "0016 - test_small" ] """
""" processed_dir = [""] """

processed_dir = ["0001 - bleach", "0002 - one piece","0003 - marvel","0004 - DC", "0005 - Love", "0006 - soma", 
                        "0007 - ishoujo", "0008 - vagabond", "0009 - vampire r","0010 - gintama", "0011 - gantz", "0012 - chainsaw",
                         "0013 - death note", "0014 - batman" ]

processed_dir = ["0001 - bleach", "0002 - one piece","0003 - marvel","0004 - DC", "0005 - Love", "0006 - soma", 
                        "0007 - ishoujo", "0008 - vagabond", "0009 - vampire r", "0010 - gintama", "0011 - gantz",
                         "0012 - chainsaw", "0013 - death note", "0014 - batman", "0015 - test"]

data_directory = 'D:\\Deletar\\pictures'
test_directory = 'G:\\Deletar\\pictures_conv'
DELTA_HUE = -1 # -1 ~ 1
DELTA_SAT = 0 # 0 < inf
DELTA_BRIGHTNESS = 0.5 # -1 ~ 1
tf.random.set_seed(42) 
SIZE = [512,512]
print(tf. __version__)

2.6.0


In [80]:
def resize_image(data):
    resized_img = tf.image.resize(
    images=data,
    size=SIZE,
    method=tf.image.ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=True,    
    name=None
    )    
    return resized_img

def adjust_hue_sat_lum(data):
    mod_img = tf.image.adjust_hue(
        data, DELTA_HUE
    )
    mod_img = tf.image.adjust_saturation(
        mod_img, DELTA_SAT
    )
    mod_img = tf.image.adjust_brightness(
        mod_img, DELTA_BRIGHTNESS
    ) 
    return mod_img
    
def remove_range(im):
    greenLower = np.array([240, 186, 200], dtype = "uint8")
    greenUpper = np.array([255, 240, 230], dtype = "uint8")
    green = cv2.inRange(im, greenLower, greenUpper)
    """ plt.imshow(im)
    plt.show() """
    print(im[1])
    green = cv2.merge([green,green,green]) - im
    return green

def save_f(img,filename,contI):
    dir_save = ""
    if (contI % 5 != 0):
        dir_save = f"{test_directory}\\train\\{filename}.jpg"
    else:
        dir_save = f"{test_directory}\\test\\{filename}.jpg"
    tf.keras.preprocessing.image.save_img(dir_save, img)
    
#if (os.path.exists(os.path.join(f"{data_directory}\\{folder}",filename+".jpg"))):    

def load_images_from_folder(folder):
    images = []
    cont = 0
    cont2 = 0    
    for folder in os.listdir(folder):
        if not folder in processed_dir:
            for filename in os.listdir(os.path.join(f"{data_directory}\\{folder}")):        
                cont2 = cont2 + 1
                if (not os.path.exists(os.path.join(f"{data_directory}\\{folder}\\",filename+".jpg"))):                        
                    try:
                        img1 = resize_image(plt.imread(os.path.join(f"{data_directory}\\{folder}\\",filename)))
                        img2 =  tf.image.rgb_to_grayscale(img1)
                        img2 = np.concatenate((img2/3,img2/3,img2/3),axis=-1)
                        img_merged = tf.concat([img1, img2], 1)
                        if img_merged is not None:
                            save_f(img_merged,filename,cont2)
                    except:
                            cont = cont + 1
                            print(f"Error in {cont}")
                print(cont2)
    return images

""" X_test = load_images_from_folder(data_directory) # join two images """


def save_bw_cl(img,filename,contI, img2):
    dir_save = ""    
    if (contI % 10 != 0):
        dir_save = f"{test_directory}\\train_new\\color\\{filename}.jpg"
        dir_save2 = f"{test_directory}\\train_new\\bw\\{filename}.jpg"
    else:
        dir_save = f"{test_directory}\\test_new\\color\\{filename}.jpg"
        dir_save2 = f"{test_directory}\\test_new\\bw\\{filename}.jpg"
    isExist = os.path.exists(dir_save)
    isExist2 = os.path.exists(dir_save2)
    """ if not isExist:
        os.makedirs(path) """
    tf.keras.preprocessing.image.save_img(dir_save, img)
    tf.keras.preprocessing.image.save_img(dir_save2, img2)

def get_bw_from_folder(folder):
    images = []
    cont = 0
    cont2 = 0    
    for folder in os.listdir(folder):
        if not folder in processed_dir:
            for filename in os.listdir(os.path.join(f"{data_directory}\\{folder}")):        
                cont2 = cont2 + 1
                
                """ if (not os.path.exists(os.path.join(f"{data_directory}\\{folder}\\",filename))): """
                               
                try:
                    img = plt.imread(os.path.join(f"{data_directory}\\{folder}\\",filename))
                    """ img3 = remove_range(img.copy()) """
                    img3 = remove_range(img)         
                    img = resize_image(img)
                    """ plt.imshow(img3)    
                    plt.show() """
                    
                    img2 = img3
                    img2 = resize_image(img2)
                    """ img2 = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY) # Input IMG is RGB
                    img2 = cv2.merge([img2,img2,img2]) """
                    
                    """ plt.imshow(img2) """
                    """ img2 = adjust_hue_sat_lum(img) """ #hue
                    if img is not None:
                        save_bw_cl(img,filename,cont2, img2)
                            
                except Exception as e:
                    cont = cont + 1                            
                    print(e)
                
                print(cont2)
                
    return images

X_test = get_bw_from_folder(data_directory)


[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [ 28   0   0]
 [ 28   0   0]
 [ 29   0   0]]
1
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
2
[[108  95 167]
 [108  95 167]
 [108  95 167]
 ...
 [208 233 229]
 [208 233 229]
 [208 233 229]]
3
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
4
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [235 211 247]
 [235 211 247]
 [235 211 247]]
5
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
6
[[125 107  85]
 [133 115  93]
 [131 114  96]
 ...
 [219 224 220]
 [220 225 221]
 [222 227 223]]
7
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
8
[[ 43  31  67]
 [ 47  35  71]
 [ 60  48  84]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
9
[[255 255 255]
 [255 255 255]
 [255 255 255]
 ...
 [255 255 255]
 [255 255 255]
 [255 255 255]]
10
